Process Melbourne Data using Methods presented in IJCAI15 Paper
==================

<a id='toc'></a>
1. [Dataset](#dataset)
  * [Simple Facts](#facts)
  * [Photo Scatter Plot](#scatter)
  * [Issues and Solutions](#issue)
1. [Results](#result)
  * [Precision, Recall and F1-score](#prf1)
  * [Transition Matrix](#transmat)

**NOTE: Please view this page via [IPython Notebook Viewer Service](http://nbviewer.ipython.org/), otherwise the within-page links may not work properly.**
<a id='dataset'></a>

1. Dataset [&#8648;](#toc)
-------------------
**Photos:**

The photos were selected from [YFCC100M dataset](http://www.referitgame.com/vicente/flickr100M/), while [Melbourne's Geo-Coordinates is 37°48′49″S 144°57′47″E](https://en.wikipedia.org/wiki/Melbourne), data in a square, from (39.5S, 140.9E) to (35.5S, 148.5E) with ```accuracy = 16``` are used, the total number of photos is 87,362.

**POIs:**

POIs are from [OpenStreeMap](http://www.openstreetmap.org/), e.g. downloading data from one of these [mirrors](http://wiki.openstreetmap.org/wiki/Planet.osm)

In [ ]:
$ wget ftp://ftp.spline.de/pub/openstreetmap/pbf/planet-latest.osm.pbf

clipping a bounding box of Melbourne: ```[140.9,-38.7, 148.5, -35.5]```, e.g. clipping using one of [these tools](http://wiki.openstreetmap.org/wiki/OSM_file_formats)

In [ ]:
$ osmconvert planet-latest.osm.pbf -b=140.9,-38.7,148.5,-35.5 -o=melbourne.osm

then filtering interested POI tags (described in the table below) from [the tag list](http://wiki.openstreetmap.org/wiki/Map_Features), the total number of POIs is 3360.

Python scripts for filtering POI tags from the clipped data using this [python library](http://imposm.org/docs/imposm.parser/latest/) is [filter_node.py](./filter_node.py), e.g.

In [ ]:
python2 filter_node.py Melb_tags.list

file ```Melb_tags.list``` is available [here](https://www.dropbox.com/sh/r50t19fb6a1m1ud/AADxEJMQWyEMJyB17ewf1-oma?dl=0).

<table>
<tr><td><b>key</b></td><td><b>values</b></td></tr>
<tr><td>```amenity```</td><td>```college, library, school, university, arts_centre, cinema, fountain, planetarium, theatre, clock, place_of_worship, ranger_station, townhall```</td></tr>
<tr><td>```building```</td><td>```farm, cathedral, chapel, church, mosque, temple, synagogue, shrine, school, stadium, university, bridge```</td></tr>
<tr><td>```geological```</td><td>```_ALL_```(indicating all values)</td></tr>
<tr><td>```historic```</td><td>```_ALL_``` (indicating all values)</td></tr>
<tr><td>```leisure```</td><td>```garden, nature_reserve, park, pitch, sports_centre, stadium, swimming_area, track, wildlife_hide```</td></tr>
<tr><td>```man_made```</td><td>```beacon, breakwater, bridge, communications_tower, embankment, dyke, groyne, lighthouse, pier, tower, windmill```</td></tr>
<tr><td>```natural```</td><td>```_ALL_``` (indicating all values)</td></tr>
<tr><td>```tourism```</td><td>```attraction, artwork, gallery, museum, picnic_site, theme_park, viewpoint, zoo```</td></tr>
<tr><td>```waterway```</td><td>```river, riverbank, stream, dam, weir, waterfall```</td></tr>
</table>

<a id='facts'></a>

### 1.1 Simple Facts [&#8648;](#toc)

Some simple facts of Melbourne data as well as data of four other cities used in ijcai15 paper are summaried in the table below.

<table>
<tr>
<td><b>City</b></td>
<td><b>&Delta;Longtitude (degree)</b></td>
<td><b>&Delta;Latitude (degree)</b></td>
<td><b>#POIs</b></td>
<td><b>#Users</b></td>
<td><b>#POI_Visits</b></td>
<td><b>#Travel_Sequences</b></td>
<td><b>Min_Distance_between_POI (km)</b></td>
<td><b>Max_Distance_between_POI (km)</b></td>
</tr>
<tr>
<td>Edinburgh</td><td>0.25</td><td>0.08</td><td>28</td><td>1,454</td><td>33,944</td><td>5,028</td><td>0.088</td><td>16.354</td>
</tr>
<tr>
<td>Toronto</td><td>0.28</td><td>0.20</td><td>29</td><td>1,395</td><td>39,419</td><td>6,057</td><td>0.147</td><td>29.655</td>
</tr>
<tr>
<td>Glasgow</td><td>0.39</td><td>0.37</td><td>27</td><td>601</td><td>11,434</td><td>2,227</td><td>0.182</td><td>45.344</td>
</tr>
<tr>
<td>Osaka</td><td>4.34</td><td>1.07</td><td>27</td><td>450</td><td>7,747</td><td>1,115</td><td>0.216</td><td>410.46</td>
</tr>
<tr>
<td><b>Melbourne</b></td><td><b>6.84</b></td><td><b>2.81</b></td><td><b>270</b></td><td><b>1,306</b></td><td><b>44,748</b></td><td><b>10,599</b></td><td><b>2.01</b></td><td><b>616.80</b></td>
</tr>
</table>

The distribution of sequence length for each city was shown below.

<table>
<tr>
<td><b>City</b></td>
<td><b>#Length 1</b></td>
<td><b>#Length 2</b></td>
<td><b>#Length 3</b></td>
<td><b>#Length 4</b></td>
<td><b>#Length 5</b></td>
<td><b>#Length 6</b></td>
<td><b>#Length 7</b></td>
<td><b>#Length 8</b></td>
<td><b>#Length 9</b></td>
<td><b>#Length 10</b></td>
<td><b>#Length 11</b></td>
<td><b>#Length 12</b></td>
<td><b>#Length 13</b></td>
</tr>
<tr><td>Edinburgh</td>
<td>3616</td><td>778</td><td>300</td><td>146</td><td>76</td><td>48</td>
<td>30</td><td>15</td><td>7</td><td>5</td><td><b>0</b></td><td>5</td><td>2</td>
</tr>
<tr><td>Toronto</td>
<td>5080</td><td>642</td><td>216</td><td>60</td><td>33</td><td>9</td>
<td>9</td><td>4</td><td>2</td><td>1</td><td><b>0</b></td><td><b>0</b></td><td>1</td>
</tr>
<tr><td>Glasgow</td>
<td>1876</td><td>239</td><td>77</td><td>20</td><td>10</td><td>2</td><td>2</td><td>1</td>
<td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td>
</tr>
<tr>
<td>Osaka</td>
<td>929</td><td>139</td><td>32</td><td>7</td><td>7</td><td>1</td><td><b>0</b></td><td><b>0</b></td>
<td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td>
</tr>
<tr>
<td><b>Melbourne</b></td>
<td><b>9817</b></td><td><b>672</b></td><td><b>81</b></td><td><b>22</b></td><td><b>4</b></td><td><b>2</b></td>
<td><b>1</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td><td><b>0</b></td>
</tr>
</table>

<a id='scatter'></a>

### 1.3 Photo Scatter Plot [&#8648;](#toc)

<a href="images/Edin_photo.png" title="Edinburgh"><img src="images/Edin_photo.png" style="width:850px;position:relative;left:-100px"></a>
<a href="images/Glas_photo.png" title="Glasgow"><img src="images/Glas_photo.png" style="width:850px;position:relative;left:-100px"></a>
<a href="images/Osak_photo.png" title="Osaka"><img src="images/Osak_photo.png" style="width:850px;position:relative;left:-100px"></a>
<a href="images/Toro_photo.png" title="Toronto"><img src="images/Toro_photo.png" style="width:850px;position:relative;left:-100px"></a>
<a href="images/Melb_photo.png" title="Melbourne OLD"><img src="images/Melb_photo.png" style="width:850px;position:relative;left:-100px"></a>
<a href="images/Melb_photo2.png" title="Melbourne NEW"><img src="images/Melb_photo2.png" style="width:850px;position:relative;left:-100px"></a>

<a id='issue'></a>

### 1.4 Issues &amp; Solutions [&#8648;](#toc)

**Q: Picking POIs is a somewhat hard task**

POIs picked manually according to photo scatter plot are much better than the results of k-means clustering/kernel density estimation, but still not good enough

**A:** With the help of [OpenStreeMap](http://www.openstreetmap.org/) and [NationalMap](http://www.nationalmap.gov.au/)/[Google Maps](https://www.google.com.au), it would be much easier to select and visualize POIs.

**Further processing the POI data:**
 1. filtering the list of POIs further, such as POIs too close, POIs with too few photos etc. (**how to?**)
 1. classify/label each POI with assistance of online maps and its original tag. (**manually?**)

NOTES:
 * Both NationalMap and GoogleMaps support headers in the first line, specify longitude and latitude in the first line when using NationalMap
 * Satellite images from GoogleMaps are better while NationalMap does not restrict the number of marks (GoogleMaps restrict <= 2000 marks each layer), marks on NationalMap will float around sometimes when dragging the map

**Q: How to deal with POIs that are too close, e.g. 0-10m?**

**A:**

**Q: POIs are generally associated with multiple labels**

how to define these labels? how to label each POI?

**A:**

**Q: Assign photo to a POI** 

if their distance is less than 200m according to paper seems not to be a good idea, as
 * if POI is something not large, e.g. buildings, 200m seems OK
 * if POI is something large, e.g. natural park, 1-2km seems to be a reasonable distance
 * but, we don't know the type of POI when assigning as picking POI and assigning photos are being done at the same time

**A:** Assign a photo to the nearest POI if the distance between the two is less than, say 500m?

**Q: Travel sequences independence assumption seems to be implausible**

Users' travel sequences are generated by splitting travel history of users if their consecutive POI visits occur more than 8 hours, while a common travelling spans several days, which could be represented by several travel sequences with dependence (e.g. user preference patterns: beach-park-shopping, beach-beach-shopping etc.)

**A:**

<a id='result'></a>

2. Results [&#8648;](#toc)
-----------
<a id='prf1'></a>

### 2.1 Precision, Recall and F1-score [&#8648;](#toc)

**Settings: Melbourne, $\eta$=0.5 with time-based user interest and POI popularity, 28/110 &asymp; 25.5% solutions are suboptimal, leave-one-out**

<table>
<tr><td><b>Recall</b></td><td><b>Precision</b></td><td><b>F1-score</b></td></tr>
<tr><td>0.735&plusmn;0.177</td><td>0.735&plusmn;0.177</td><td>0.735&plusmn;0.177</td></tr>
</table>

<table>
<tr><td><b>Value(Recall/Precision/F1-score)</b></td><td>1.0</td><td>0.75</td><td>0.67</td><td>0.60</td><td>0.57</td><td>0.50</td><td>0.40</td></tr>
<tr><td><b>Frequency</b></td><td>30/110</td><td>7/110</td><td>54/110</td><td>2/110</td><td>1/110</td><td>14/110</td><td>2/110</td></tr> 
</table>

Box plot of Recall, Precision and F1-score
<a href="images/Melb_bp_eta0.png" title="Melbourne"><img src="images/Melb_bp_eta0.png" style="width:400px;position:relative;left:-100px"></a>

**Settings: Melbourne, $\eta$=0.0 with POI popularity only, 29/110 &asymp; 26.4% solutions are suboptimal, leave-one-out**

<table>
<tr><td><b>Recall</b></td><td><b>Precision</b></td><td><b>F1-score</b></td></tr>
<tr><td>0.732&plusmn;0.176</td><td>0.732&plusmn;0.176</td><td>0.732&plusmn;0.176</td></tr>
</table>

<table>
<tr><td><b>Value(Recall/Precision/F1-score)</b></td><td>1.0</td><td>0.83</td><td>0.75</td><td>0.67</td><td>0.60</td><td>0.57</td><td>0.50</td><td>0.40</td></tr>
<tr><td><b>Frequency</b></td><td>29/110</td><td>1/110</td><td>6/110</td><td>55/110</td><td>2/110</td><td>1/110</td><td>14/110</td><td>2/110</td></tr> 
</table>

Box plot of Recall, Precision and F1-score
<a href="images/Melb_bp_eta05_time.png" title="Melbourne"><img src="images/Melb_bp_eta05_time.png" style="width:400px;position:relative;left:-100px"></a>

<a id='transmat'></a>

### 2.2 Transition Matrix [&#8648;](#toc)

Transition matrix for *recommended* sequences, $\eta$ = 0.5:

<table>
<tr><td></td><td><b>Beach</b></td><td><b>Cultural</b></td><td><b>Education</b></td><td><b>Forest</b></td><td><b>Leisure</b></td><td><b>ManMade</b></td><td><b>Natural</b></td><td><b>Park</b></td><td><b>Religion</b></td><td><b>Shopping</b></td><td><b>WaterBody</b></td></tr>
<tr><td><b>Beach</b></td><td>0.176</td><td>0.020</td><td>0.078</td><td>0.000</td><td>0.020</td><td>0.000</td><td>0.000</td><td>0.196</td><td>0.216</td><td>0.294</td><td>0.000</td></tr>
<tr><td><b>Cultural</b></td><td>0.429</td><td>0.143</td><td>0.000</td><td>0.000</td><td>0.143</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.143</td><td>0.143</td><td>0.000</td></tr>
<tr><td><b>Education</b></td><td>0.353</td><td>0.000</td><td>0.118</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.059</td><td>0.176</td><td>0.294</td><td>0.000</td></tr>
<tr><td><b>Forest</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>1.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td></tr>
<tr><td><b>Leisure</b></td><td>0.100</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.100</td><td>0.000</td><td>0.100</td><td>0.000</td><td>0.100</td><td>0.600</td><td>0.000</td></tr>
<tr><td><b>ManMade</b></td><td>0.000</td><td>0.000</td><td>1.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td></tr>
<tr><td><b>Natural</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>1.000</td><td>0.000</td></tr>
<tr><td><b>Park</b></td><td>0.316</td><td>0.000</td><td>0.026</td><td>0.000</td><td>0.079</td><td>0.000</td><td>0.000</td><td>0.053</td><td>0.132</td><td>0.395</td><td>0.000</td></tr>
<tr><td><b>Religion</b></td><td>0.240</td><td>0.160</td><td>0.000</td><td>0.040</td><td>0.040</td><td>0.040</td><td>0.000</td><td>0.080</td><td>0.120</td><td>0.280</td><td>0.000</td></tr>
<tr><td><b>Shopping</b></td><td>0.186</td><td>0.010</td><td>0.039</td><td>0.010</td><td>0.059</td><td>0.000</td><td>0.010</td><td>0.167</td><td>0.069</td><td>0.451</td><td>0.000</td></tr>
<tr><td><b>WaterBody</b></td><td>0.143</td><td>0.143</td><td>0.286</td><td>0.143</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.143</td><td>0.000</td><td>0.143</td><td>0.000</td></tr>
</table>

Transition matrix for *recommended* sequences, $\eta$ = 0.0:

<table>
<tr><td></td><td><b>Beach</b></td><td><b>Cultural</b></td><td><b>Education</b></td><td><b>Forest</b></td><td><b>Leisure</b></td><td><b>ManMade</b></td><td><b>Natural</b></td><td><b>Park</b></td><td><b>Religion</b></td><td><b>Shopping</b></td><td><b>WaterBody</b></td></tr>
<tr><td><b>Beach</b></td><td>0.143</td><td>0.020</td><td>0.061</td><td>0.000</td><td>0.020</td><td>0.000</td><td>0.000</td><td>0.265</td><td>0.204</td><td>0.286</td><td>0.000</td>
</tr>
<tr><td><b>Cultural</b></td><td>0.571</td><td>0.143</td><td>0.000</td><td>0.000</td><td>0.143</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.143</td><td>0.000</td><td>0.000</td>
</tr>
<tr><td><b>Education</b></td><td>0.278</td><td>0.000</td><td>0.056</td><td>0.000</td><td>0.056</td><td>0.000</td><td>0.000</td><td>0.056</td><td>0.167</td><td>0.389</td><td>0.000</td>
</tr>
<tr><td><b>Forest</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>1.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td>
</tr>
<tr><td><b>Leisure</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.125</td><td>0.000</td><td>0.062</td><td>0.125</td><td>0.062</td><td>0.625</td><td>0.000</td>
</tr>
<tr><td><b>ManMade</b></td><td>0.000</td><td>0.000</td><td>0.333</td><td>0.333</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.333</td><td>0.000</td><td>0.000</td><td>0.000</td>
</tr>
<tr><td><b>Natural</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>1.000</td><td>0.000</td>
</tr>
<tr><td><b>Park</b></td><td>0.342</td><td>0.000</td><td>0.105</td><td>0.000</td><td>0.105</td><td>0.026</td><td>0.000</td><td>0.026</td><td>0.053</td><td>0.316</td><td>0.026</td>
</tr>
<tr><td><b>Religion</b></td><td>0.174</td><td>0.174</td><td>0.000</td><td>0.043</td><td>0.043</td><td>0.043</td><td>0.000</td><td>0.087</td><td>0.174</td><td>0.261</td><td>0.000</td>
</tr>
<tr><td><b>Shopping</b></td><td>0.219</td><td>0.010</td><td>0.042</td><td>0.000</td><td>0.094</td><td>0.000</td><td>0.010</td><td>0.115</td><td>0.083</td><td>0.427</td><td>0.000</td>
</tr>
<tr><td><b>WaterBody</b></td><td>0.125</td><td>0.125</td><td>0.250</td><td>0.125</td><td>0.000</td><td>0.125</td><td>0.000</td><td>0.250</td><td>0.000</td><td>0.000</td><td>0.000</td>
</tr>
</table>

Transition matrix for *actual* sequences:

<table>
<tr><td></td><td><b>Beach</b></td><td><b>Cultural</b></td><td><b>Education</b></td><td><b>Forest</b></td><td><b>Leisure</b></td><td><b>ManMade</b></td><td><b>Natural</b></td><td><b>Park</b></td><td><b>Religion</b></td><td><b>Shopping</b></td><td><b>WaterBody</b></td></tr>
<tr><td><b>Beach</b></td><td>0.454</td><td>0.008</td><td>0.008</td><td>0.000</td><td>0.042</td><td>0.000</td><td>0.000</td><td>0.042</td><td>0.092</td><td>0.353</td><td>0.000</td></tr>
<tr><td><b>Cultural</b></td><td>0.027</td><td>0.027</td><td>0.054</td><td>0.000</td><td>0.108</td><td>0.000</td><td>0.000</td><td>0.189</td><td>0.189</td><td>0.378</td><td>0.027</td></tr>
<tr><td><b>Education</b></td><td>0.038</td><td>0.000</td><td>0.170</td><td>0.000</td><td>0.057</td><td>0.000</td><td>0.000</td><td>0.113</td><td>0.038</td><td>0.528</td><td>0.057</td></tr>
<tr><td><b>Forest</b></td><td>0.000</td><td>0.100</td><td>0.000</td><td>0.500</td><td>0.000</td><td>0.000</td><td>0.100</td><td>0.100</td><td>0.000</td><td>0.000</td><td>0.200</td></tr>
<tr><td><b>Leisure</b></td><td>0.045</td><td>0.164</td><td>0.060</td><td>0.000</td><td>0.119</td><td>0.000</td><td>0.015</td><td>0.075</td><td>0.060</td><td>0.448</td><td>0.015</td></tr>
<tr><td><b>ManMade</b></td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.500</td><td>0.500</td><td>0.000</td></tr>
<tr><td><b>Natural</b></td><td>0.000</td><td>0.000</td><td>0.091</td><td>0.000</td><td>0.000</td><td>0.000</td><td>0.818</td><td>0.091</td><td>0.000</td><td>0.000</td><td>0.000</td></tr>
<tr><td><b>Park</b></td><td>0.031</td><td>0.087</td><td>0.094</td><td>0.008</td><td>0.055</td><td>0.000</td><td>0.000</td><td>0.039</td><td>0.055</td><td>0.614</td><td>0.016</td></tr>
<tr><td><b>Religion</b></td><td>0.091</td><td>0.073</td><td>0.073</td><td>0.018</td><td>0.127</td><td>0.000</td><td>0.000</td><td>0.073</td><td>0.073</td><td>0.436</td><td>0.036</td></tr>
<tr><td><b>Shopping</b></td><td>0.137</td><td>0.045</td><td>0.033</td><td>0.002</td><td>0.073</td><td>0.000</td><td>0.000</td><td>0.104</td><td>0.043</td><td>0.545</td><td>0.017</td></tr>
<tr><td><b>WaterBody</b></td><td>0.172</td><td>0.000</td><td>0.138</td><td>0.172</td><td>0.069</td><td>0.000</td><td>0.000</td><td>0.069</td><td>0.000</td><td>0.345</td><td>0.034</td></tr>
</table>